<a href="https://colab.research.google.com/github/chclau/MIF_Gen/blob/master/square_tr_cnn_v3_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import math
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle


# DML CNN section
from __future__ import absolute_import, division, print_function, unicode_literals

# Import TensorFlow and TensorFlow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds
tf.logging.set_verbosity(tf.logging.ERROR)

# -----------------------------------------------------------------------------------------------------------------------------------
# Generate squares and triangles training database
img_size = 12;
train_size = 600;
test_size  = np.int8(train_size/5);
sq_train   = np.zeros((img_size, img_size, train_size));
tr_train   = np.zeros((img_size, img_size, train_size));
sq_test    = np.zeros((img_size, img_size, train_size));
tr_test    = np.zeros((img_size, img_size, train_size));

for k in range(train_size):
  x0 = np.int8(3 * np.random.random_sample())+1;   # random origin x coord
  y0 = np.int8(3 * np.random.random_sample())+1;   # random origin y coord
  w = np.int8(img_size/3 * np.random.random_sample()+img_size/3);   # random size

  # square
  for i in range(w):
    for j in range(w):
      sq_train[y0+i, x0+j, k] = 1;
  
  # triangle      
  for i in range(w):
    for j in range(i):
      tr_train[y0+i, x0+j, k] = 1;
  
#plt.imshow(sq[:,:,11], cmap=plt.cm.binary)
#plt.imshow(tr[:,:,11], cmap=plt.cm.binary)

# Generate squares and triangles validation database
for k in range(test_size):
  x0 = np.int8(3 * np.random.random_sample()+img_size/4);   # random origin x coord for test images
  y0 = np.int8(3 * np.random.random_sample()+img_size/4);   # random origin y coord for test images
  w = np.int8(img_size/3 * np.random.random_sample()+img_size/3);   # random size

  # square
  for i in range(w):
    for j in range(w):
      sq_test[y0+i, x0+j, k] = 1;
  
  # triangle      
  for i in range(w):
    for j in range(i):
      tr_test[y0+i, x0+j, k] = 1;
      
# Generate images database
train_images = [];
for i in range(train_size):
  train_images.append([sq_train[:,:,i], 0]);
  train_images.append([tr_train[:,:,i], 1]);
  shuffle(train_images);

test_images = [];
for i in range(test_size):
  test_images.append([sq_test[:,:,i], 0]);
  test_images.append([tr_test[:,:,i], 1]);
  shuffle(test_images);
  

# -----------------------------------------------------------------------------------------------------------------------------------
# Model generate
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(img_size, img_size,1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2,   activation=tf.nn.softmax)
])

# Model compile
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

tr_img_data = np.array([i[0] for i in train_images]).reshape(-1,img_size,img_size,1)
tr_lbl_data = np.array([i[1] for i in train_images])
tst_img_data = np.array([i[0] for i in test_images]).reshape(-1,img_size,img_size,1)
tst_lbl_data = np.array([i[1] for i in test_images])

# Model fit
model.fit(tr_img_data, tr_lbl_data, epochs=30, batch_size=128)
model.summary();
test_loss, test_accuracy = model.evaluate(tst_img_data, tst_lbl_data)


IndentationError: ignored

In [31]:
# -----------------------------------------------------------------------------------------------------------------------------------
# Model predict
x0 = 1; 
y0 = 1; 
w  = 5;
pred   = np.zeros((img_size, img_size, 1));
for i in range(w):
  for j in range(w):
    pred[y0+i, x0+j, 0] = 1;

pred_img_data = pred.reshape(-1,img_size,img_size,1)
model.predict(pred_img_data)

array([[1.0000000e+00, 4.8972963e-09]], dtype=float32)